This example showcases best practices for loading data into a Syd Viewer. 

It gives three examples for different contexts related to how much data is loaded and how long it takes to load. 

In [1]:
try:
    import google.colab
    # We're in Colab
    !pip install git+https://github.com/landoskape/syd.git

except ImportError:
    pass

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from syd import Viewer

Suppose your dataset is composed of electrophysiology recordings from 3 mice, where each mouse has a different number of sesssions, and each session has a different number of neurons. You want to build a viewer to choose the mouse, then choose the session, and then view a particular neuron from within that session. But the viewer will break if you try to index to session 5 for mouse 2 but mouse 2 only has 4 sessions! How can you structure callbacks that make sure your syd parameters are always valid?


To show how to do this, we'll start with a toy example as described above.

In [2]:
# Let's create a toy dataset to create syd viewers from.
num_mice = 3
num_sessions_per_mouse = [3, 4, 5]
num_neurons_per_session = []
num_timepoints = 1000
for imouse in range(num_mice):
    c_num_sessions = num_sessions_per_mouse[imouse]
    c_num_neurons = np.random.randint(100, 1000, c_num_sessions)
    num_neurons_per_session.append(c_num_neurons)

# Create a toy dataset
mice_names = [f"Mouse {imouse}" for imouse in range(num_mice)]
session_data = {}
for imouse, mouse_name in enumerate(mice_names):
    c_num_sessions = num_sessions_per_mouse[imouse]
    c_session_data = []
    for isession in range(c_num_sessions):
        c_num_neurons = num_neurons_per_session[imouse][isession]
        c_session_data.append(np.random.randn(c_num_neurons, num_timepoints))
    session_data[mouse_name] = c_session_data

# Build our loading methods
def get_num_sessions(mouse_name):
    return len(session_data[mouse_name])

def get_num_neurons(mouse_name, session_index):
    return len(session_data[mouse_name][session_index])

def get_data(mouse_name, session_index, neuron_index):
    return session_data[mouse_name][session_index][neuron_index]

# And make a simple plotting method.
def plot_the_data(data):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(data)
    return fig

In [ ]:
class MouseViewer(Viewer):
    def __init__(self, mice_names):
        self.mice_names = mice_names

        self.add_selection("mouse", options=list(mice_names))

        # We don't know how many sessions or neurons to pick from yet!
        self.add_integer("session", min=0, max=1)
        self.add_integer("neuron", min=0, max=1)
        
        # Any time the mouse changes, update the sessions to pick from
        self.on_change("mouse", self.update_mouse)

        # Any time the session changes, update the neurons to pick from
        self.on_change("session", self.update_session)

        # Since we built callbacks for setting the range of the session
        # and neuron parameters, we can use them here!
        # To get the state, we can use self.state, which is the current
        # state of the viewer (in the init function, it'll just be the
        # default value for each parameter you've added already).
        self.update_mouse(self.state)

    def update_mouse(self, state):
        # Pseudo code for getting the number of sessions for a given mouse
        num_sessions = get_num_sessions(state["mouse"])

        # Now we update the number of sessions to pick from
        self.update_integer("session", max=num_sessions - 1)

        # Now we need to update the neurons to choose from ....
        # But! Updating the session parameter might trigger a change to the
        # session value. So, instead of using the state dictionary that was
        # passed into the function, we can get the ~NEW~ state dictionary like this:
        new_state = self.state

        # Then perform the session update callback!
        self.update_session(new_state)

    def update_session(self, state):
        num_neurons = get_num_neurons(state["mouse"], state["session"])

        # Now we update the number of neurons to pick from
        self.update_integer("neuron", max=num_neurons - 1)

    def plot(self, state):
        data = get_data(state["mouse"], state["session"], state["neuron"])
        fig = plot_the_data(data)
        return fig

# Now we can create a viewer and deploy it
viewer = MouseViewer(mice_names)
viewer.show()

# To see how the callbacks work --- try increasing session or neuron to the maximum value and then changing the mouse.
# The maximum value of the slider will update, which will either change the session/neuron value to the maximum permitted,
# or it will leave the value but change the slider range so it'll be at a lower relative position within the slider.